In [100]:
import pandas as pd
import numpy as np
import math

import itertools
from tqdm import tqdm

from datetime import datetime, date
import pandas_market_calendars as mcal

from joblib import Parallel, delayed

In [101]:
daily_end_date = date.today().strftime('%Y%m%d')

krx = mcal.get_calendar('XKRX')
nyse = mcal.get_calendar('NYSE')

krx_trading_day = krx.schedule(start_date='20150101', end_date=daily_end_date).index.tolist()
nyse_trading_day = nyse.schedule(start_date='20150101', end_date=daily_end_date).index.tolist()
        
all_trading_days_list = sorted(set(krx_trading_day + nyse_trading_day))

all_trading_days_dict = {}

for i, day in enumerate(all_trading_days_list):
    all_trading_days_dict[day] = i+1

/Users/admin/anaconda3/lib/python3.10/site-packages/pandas_market_calendars/market_calendar.py:144: UserWarning: ['break_start', 'break_end'] are discontinued, the dictionary `.discontinued_market_times` has the dates on which these were discontinued. The times as of those dates are incorrect, use .remove_time(market_time) to ignore a market_time.
  warnings.warn(f"{list(discontinued.keys())} are discontinued, the dictionary"


In [102]:
day_idx = all_trading_days_dict[datetime.strptime('20230602', '%Y%m%d')]

In [7]:
name_df = pd.read_csv('/Users/admin/Desktop/life_web_ui/(kospi+russell)_ticker_name_sector_description.csv')

In [33]:
sector1 = list(set(name_df['Sector'].values.tolist()))
sector2 = ['산업재','임의소비재', '에너지', '소재', '헬스케어', '통신서비스', '부동산', '유틸리티', '필수소비재', '금융', '테크놀로지']

In [9]:
to_name_dict = {}
for i,j in zip(name_df['Name_kor'], name_df['MySQL']):
    to_name_dict[j] = i
name1_list = [to_name_dict[x] for x in ticker1_list]
name2_list = [to_name_dict[x] for x in ticker2_list]

In [38]:
to_sector_dict = {}
for i,j in zip(sector1, sector2):
    to_sector_dict[i] = j
sector_kor = [to_sector_dict[x] for x in name_df['Sector'].tolist()]

In [40]:
name_df['Sector_kor'] = sector_kor

In [45]:
name_df = name_df[['Ticker', 'Bloomberg', 'MySQL', 'Name_kor', 'Name', 'Sector_kor', 'Sector',
       'Industry_Group', 'Industry', 'Sub_Industry', 'Description']]

In [47]:
name_df.to_csv('/Users/admin/Desktop/life_web_ui/(kospi+russell)_ticker_name_sector_description.csv', index=None)

In [66]:
sector1_list = [name_df[name_df['MySQL'] == ticker]['Sector_kor'].values[0] for ticker in ticker1_list]

In [67]:
sector2_list = [name_df[name_df['MySQL'] == ticker]['Sector_kor'].values[0] for ticker in ticker2_list]

In [103]:
members_list_df = pd.read_csv('/Users/admin/Desktop/life_web_ui/members_list_by_date.csv', index_col = 0)
members_list = sorted(eval(members_list_df.iloc[-1]['members_list']))
ks_mems_list = []
rs_mems_list = []
for mems in members_list:
    if mems.endswith('KS'):
        ks_mems_list.append(mems)
    else:
        rs_mems_list.append(mems)
ks_pairs_list = list(itertools.combinations(ks_mems_list, 2))
ks_rs_pairs_list = list(itertools.product(ks_mems_list, rs_mems_list))
all_pairs_list = ks_pairs_list + ks_rs_pairs_list

ticker1_list = []
ticker2_list = []
for pair in tqdm(all_pairs_list):
    ticker1_list.append(pair[0])
    ticker2_list.append(pair[1])

100%|██████████████████████████████| 601700/601700 [00:00<00:00, 5466628.36it/s]


In [91]:
df = pd.read_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_len60.csv', index_col='ranking')

In [99]:
df[~df['historical_corr'].isnull()]

,name1,name2,corr,corr_level,historical_corr,sector1,sector2,ticker1,ticker2
ranking,,,,,,,,,
1,기아,현대차,0.803797,★★★,"[0.7030160000000001, 0.6995682000000001, 0.703...",임의소비재,임의소비재,000270_KS,005380_KS
2,아모레G,아모레퍼시픽,0.795712,★★☆,"[0.7588801999999999, 0.7606216, 0.757520999999...",필수소비재,필수소비재,002790_KS,090430_KS
3,GS건설,대우건설,0.725157,★★☆,"[0.8017642, 0.8024986000000001, 0.791607200000...",산업재,산업재,006360_KS,047040_KS
4,현대해상,DB손해보험,0.706705,★★☆,"[0.7133584000000001, 0.7148432, 0.7079074, 0.7...",금융,금융,001450_KS,005830_KS
5,HD한국조선해양,삼성중공업,0.688938,★★☆,"[0.7096186, 0.7027015999999999, 0.719379999999...",산업재,산업재,009540_KS,010140_KS
...,...,...,...,...,...,...,...,...,...
600896,아모레G,비글라리 홀딩스 Class B,-0.272802,☆☆☆,"[-0.1736338, -0.1809386, -0.1822984, -0.20338,...",필수소비재,임의소비재,002790_KS,BH_UN
600897,대한유화,DR 호턴,-0.277994,☆☆☆,"[-0.2908168, -0.2855690000000001, -0.312016, -...",소재,임의소비재,006650_KS,DHI_UN
600898,하이브,스티치 픽스,-0.287427,☆☆☆,"[-0.2840594000000001, -0.2896178, -0.2752912, ...",통신서비스,임의소비재,352820_KS,SFIX_UW


In [114]:
def add_describe(ticker1, ticker2, window_size):
    try:
        df = pd.read_csv(f'/Users/admin/Desktop/life_web_ui/database/results{window_size}/{ticker1}_{ticker2}_corr_len{window_size}.csv', index_col='date_keys')
        max_, min_, avg_ = df['corr_w_sum'].max(), df['corr_w_sum'].min(), df['corr_w_sum'].mean()
    except:
        max_, min_, avg_ = np.nan, np.nan, np.nan
    return(avg_,max_,min_)

In [115]:
while len(tqdm._instances) > 0:
    tqdm._instances.pop().close()

result_list = []
for window_size in tqdm([20,40,60]):
    with Parallel(n_jobs=-1) as parallel:
        results = parallel(delayed(add_describe)(pair[0], pair[1], window_size) for pair in tqdm(all_pairs_list))
    result_list.append(results)

 16%|█████▍                            | 97280/601700 [00:29<02:31, 3324.05it/s]


 33%|██████████▊                      | 196608/601700 [01:00<02:15, 2984.92it/s]


 49%|████████████████▏                | 295936/601700 [01:31<01:40, 3056.46it/s]


 66%|█████████████████████▋           | 395264/601700 [02:05<01:13, 2814.53it/s]


 82%|███████████████████████████▏     | 494592/601700 [02:37<00:34, 3134.17it/s]


...

 47%|███████████████▍                 | 282608/601700 [01:48<02:02, 2600.45it/s]


 63%|████████████████████▉            | 381936/601700 [02:27<01:30, 2439.50it/s]


 80%|██████████████████████████▍      | 481264/601700 [03:07<00:47, 2513.34it/s]


 96%|███████████████████████████████▊ | 580592/601700 [03:42<00:07, 3012.83it/s]


100%|████████████████████████████████████████████| 3/3 [10:34<00:00, 211.58s/it]


In [123]:
all_pairs_dict = {}
for i, j in enumerate(all_pairs_list):
    all_pairs_dict[j] = i

In [130]:
for window_size, result in zip([20,40,60], result_list):
    df = pd.read_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_len{window_size}.csv', index_col='ranking')
    avg_list = []
    max_list = []
    min_list = []
    for _, row in tqdm(df.iterrows()):
        pair = (row['ticker1'], row['ticker2'])
        idx = all_pairs_dict[pair]
        avg_list.append(result[idx][0])
        max_list.append(result[idx][1])
        min_list.append(result[idx][2])
    df['avg_corr'] = avg_list
    df['max_corr'] = max_list
    df['min_corr'] = min_list
    df.to_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_len{window_size}.csv')

601700it [00:08, 67858.68it/s]
601700it [00:08, 67799.20it/s]
601700it [00:08, 67707.54it/s]


In [135]:
def replace_ratings(text):
    if text == '★★★':
        return '⭐️⭐️⭐️'
    elif text == '★★☆':
        return '⭐️⭐️'
    elif text == '★☆☆':
        return '⭐️'
    elif text == '☆☆☆':
        return '❌'
    elif text == '에러':
        return '❓'
    else:
        return None

In [136]:
for window_size, result in zip([20,40,60], result_list):
    df = pd.read_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_len{window_size}.csv', index_col='ranking')
    df['avg_corr'] = df['avg_corr'].round(3)
    df['max_corr'] = df['max_corr'].round(3)
    df['min_corr'] = df['min_corr'].round(3)
    
    df = df[['name1', 'name2', 'corr', 'corr_level', 'historical_corr', 'avg_corr', 'max_corr', 'min_corr', 'sector1', 
       'sector2', 'ticker1', 'ticker2']]
    
    df['corr_level'] = [replace_ratings(x) for x in df['corr_level']]
    
    df.to_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_len{window_size}.csv')

In [140]:
pd.read_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_len60.csv', index_col='ranking')

,name1,name2,corr,corr_level,historical_corr,avg_corr,max_corr,min_corr,sector1,sector2,ticker1,ticker2
ranking,,,,,,,,,,,,
1,기아,현대차,0.803797,⭐️⭐️⭐️,"[0.7030160000000001, 0.6995682000000001, 0.703...",0.517,0.857,0.168,임의소비재,임의소비재,000270_KS,005380_KS
2,아모레G,아모레퍼시픽,0.795712,⭐️⭐️,"[0.7588801999999999, 0.7606216, 0.757520999999...",0.614,0.871,0.223,필수소비재,필수소비재,002790_KS,090430_KS
3,GS건설,대우건설,0.725157,⭐️⭐️,"[0.8017642, 0.8024986000000001, 0.791607200000...",0.429,0.886,-0.026,산업재,산업재,006360_KS,047040_KS
4,현대해상,DB손해보험,0.706705,⭐️⭐️,"[0.7133584000000001, 0.7148432, 0.7079074, 0.7...",0.498,0.814,0.045,금융,금융,001450_KS,005830_KS
5,HD한국조선해양,삼성중공업,0.688938,⭐️⭐️,"[0.7096186, 0.7027015999999999, 0.719379999999...",0.512,0.894,0.041,산업재,산업재,009540_KS,010140_KS
...,...,...,...,...,...,...,...,...,...,...,...,...
601696,F&F,스타 홀딩스,NaN,에러,NaN,NaN,NaN,NaN,임의소비재,부동산,383220_KS,STHO_UQ
601697,SK스퀘어,크레인,NaN,에러,NaN,NaN,NaN,NaN,산업재,산업재,402340_KS,CR_UN
601698,SK스퀘어,나이프 리버,NaN,에러,NaN,NaN,NaN,NaN,산업재,소재,402340_KS,KNF_UN


In [144]:
ticker_info_df = pd.read_csv('/Users/admin/Desktop/life_web_ui/(kospi+russell)_ticker_name_sector_description.csv')

In [152]:
ticker_info_df[ticker_info_df['MySQL'].isin(['000660_KS', 'MU_UW'])][['Description' ]].iloc[0,0]

'SK hynix Inc. provides products and services for the electronic components industries. The Company manufactures semiconductors such as dynamic random access memory (DRAM), NAND flash memory, and static random access memory (SRAM) chips.'

In [134]:


       'sector2', 'ticker1', 'ticker2', 'avg_corr', 'max_corr', 'min_corr']]

Index(['name1', 'name2', 'corr', 'corr_level', 'historical_corr', 'sector1',
       'sector2', 'ticker1', 'ticker2', 'avg_corr', 'max_corr', 'min_corr'],
      dtype='object')

In [118]:
len(result_list)

3

In [119]:
result_list[0]

601700

,name1,name2,corr,corr_level,historical_corr,sector1,sector2,ticker1,ticker2
ranking,,,,,,,,,
1,기아,현대차,0.803797,★★★,"[0.7030160000000001, 0.6995682000000001, 0.703...",임의소비재,임의소비재,000270_KS,005380_KS
2,아모레G,아모레퍼시픽,0.795712,★★☆,"[0.7588801999999999, 0.7606216, 0.757520999999...",필수소비재,필수소비재,002790_KS,090430_KS
3,GS건설,대우건설,0.725157,★★☆,"[0.8017642, 0.8024986000000001, 0.791607200000...",산업재,산업재,006360_KS,047040_KS
4,현대해상,DB손해보험,0.706705,★★☆,"[0.7133584000000001, 0.7148432, 0.7079074, 0.7...",금융,금융,001450_KS,005830_KS
5,HD한국조선해양,삼성중공업,0.688938,★★☆,"[0.7096186, 0.7027015999999999, 0.719379999999...",산업재,산업재,009540_KS,010140_KS
...,...,...,...,...,...,...,...,...,...
601696,F&F,스타 홀딩스,NaN,에러,NaN,임의소비재,부동산,383220_KS,STHO_UQ
601697,SK스퀘어,크레인,NaN,에러,NaN,산업재,산업재,402340_KS,CR_UN
601698,SK스퀘어,나이프 리버,NaN,에러,NaN,산업재,소재,402340_KS,KNF_UN


In [112]:
result_list

[[(0.05415000073903002, 0.7108688, -0.311053),
  (0.08457224120092378, 0.7682408000000001, -0.369764)],
 [(0.05077383813519813, 0.465145, -0.1276802),
  (0.08578308652680652, 0.5672434, -0.158804)],
 [(0.04692340216470587, 0.353005, -0.13122),
  (0.08815542879999999, 0.4841305999999999, -0.1397792)]]

In [76]:
def read_atten_day(ticker1, ticker2, window_size, idx):
    try:
        df = pd.read_csv(f'/Users/admin/Desktop/life_web_ui/database/results{window_size}/{ticker1}_{ticker2}_corr_len{window_size}.csv', index_col='date_keys')
        corr_list = df.loc[idx-min(10, len(df)):idx, 'corr_w_sum'].tolist()
        corr = corr_list[-1]
    except:
        corr_list = None
        corr = np.nan
    return corr, corr_list
    
def map_corr_level(corr):
    if pd.isna(corr):
        return '에러'
    elif abs(corr) >= 0.8:
        return '★★★'
    elif abs(corr) >= 0.6:
        return '★★☆'
    elif abs(corr) >= 0.4:
        return '★☆☆'
    else:
        return '☆☆☆'
          
while len(tqdm._instances) > 0:
    tqdm._instances.pop().close()
    
for window_size in tqdm([20,40,60]):
    with Parallel(n_jobs=-1) as parallel:
        results = parallel(delayed(read_atten_day)(pair[0], pair[1], window_size, day_idx) for pair in tqdm(all_pairs_list))

    corr_list = [x[0] for x in results]
    recent_corr_list = [x[1] for x in results]

    ranking_df = pd.DataFrame(columns=['name1', 'name2', 'corr', 'corr_level', 'historical_corr', 'sector1', 'sector2', 'ticker1', 'ticker2'])

    ranking_df['name1'] = name1_list
    ranking_df['name2'] = name2_list
    ranking_df['corr'] = corr_list
    ranking_df['corr_level'] = ranking_df['corr'].apply(map_corr_level)
    ranking_df['historical_corr'] = recent_corr_list
    ranking_df['sector1'] = sector1_list
    ranking_df['sector2'] = sector2_list
    ranking_df['ticker1'] = ticker1_list
    ranking_df['ticker2'] = ticker2_list
    ranking_df = ranking_df.sort_values(by='corr', ascending=False)
    ranking_df.index = range(1, 1+len(ranking_df))
    ranking_df.index.name = 'ranking'
    ranking_df.to_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_len{window_size}.csv')

# ranking_ksrs_df = ranking_df[ranking_df['ticker2'].str[-2:] != 'KS']
# ranking_ksrs_df.index = range(1,1+len(ranking_ksrs_df))
# ranking_ksrs_df.index.name = 'ranking'
# ranking_ksrs_df.to_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_ksrs_corr_{window_size}.csv')

# plus_ranking_ksrs_df = ranking_ksrs_df[ranking_ksrs_df['corr'] >= 0]
# plus_ranking_ksrs_df.to_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_ksrs_plus_corr_{window_size}.csv')

# minus_ranking_ksrs_df = ranking_ksrs_df[ranking_ksrs_df['corr'] <= 0]
# minus_ranking_ksrs_df = minus_ranking_ksrs_df.sort_values(by='corr', ascending=True)
# minus_ranking_ksrs_df.index = range(1,1+len(minus_ranking_ksrs_df))
# minus_ranking_ksrs_df.to_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_ksrs_minus_corr_{window_size}.csv')

# ranking_df = ranking_df[['ticker1', 'ticker2', 'corr']]

# ranking_ksks_df = ranking_df[ranking_df['ticker2'].str[-2:] == 'KS']
# ranking_ksks_df.index = range(1,1+len(ranking_ksks_df))
# ranking_ksks_df.index.name = 'ranking'
# ranking_ksks_df.to_csv(f'/Users/admin/Desktop/life_web_ui/database/ranking/results_ksks_corr_{window_size}.csv')

 16%|█████▍                            | 96776/601700 [00:30<02:24, 3495.16it/s]


 33%|██████████▊                      | 196104/601700 [01:00<02:13, 3029.94it/s]


 49%|████████████████▏                | 295432/601700 [01:30<01:34, 3226.62it/s]


 66%|█████████████████████▋           | 394760/601700 [02:03<01:10, 2939.46it/s]


 82%|███████████████████████████      | 494088/601700 [02:34<00:32, 3264.56it/s]


...

 46%|███████████████▎                 | 279024/601700 [01:27<01:46, 3025.47it/s]


 63%|████████████████████▊            | 378352/601700 [02:01<01:17, 2871.82it/s]


 79%|██████████████████████████▏      | 477680/601700 [02:35<00:39, 3140.13it/s]


 96%|███████████████████████████████▋ | 577008/601700 [03:05<00:06, 4064.02it/s]


100%|████████████████████████████████████████████| 3/3 [09:42<00:00, 194.14s/it]


검증용 코드

In [6]:
len(corr_list)

601700

In [7]:
ranking_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601700 entries, 1212 to 601248
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   ticker1  601700 non-null  object 
 1   ticker2  601700 non-null  object 
 2   corr     600900 non-null  float64
dtypes: float64(1), object(2)
memory usage: 18.4+ MB


In [8]:
nan_df = ranking_df[ranking_df['corr'].isnull()]
nan_df = nan_df.reset_index(drop=True)

In [9]:
nan_df

,ticker1,ticker2,corr
0,000080_KS,CR_UN,NaN
1,000080_KS,KNF_UN,NaN
2,000080_KS,MSGE_UN,NaN
3,000080_KS,STHO_UQ,NaN
4,000100_KS,CR_UN,NaN
...,...,...,...
795,383220_KS,STHO_UQ,NaN
796,402340_KS,CR_UN,NaN
797,402340_KS,KNF_UN,NaN
798,402340_KS,MSGE_UN,NaN


In [111]:
window_size = 60
idx = 0
ticker1 = nan_df.loc[idx, 'ticker1']
ticker2 = nan_df.loc[idx, 'ticker2']
print(ticker1, ticker2)

df = pd.read_csv(f'/Users/admin/Desktop/life_web_ui/database/results{window_size}/{ticker1}_{ticker2}_corr_len{window_size}.csv', index_col='date_keys')

000080_KS 001440_KS


In [112]:
df

,corr_px,corr_log_r(win),corr_w_sum
date_keys,,,
59,-0.434517,-0.256681,-0.090469
60,-0.443334,-0.310765,-0.116569
61,-0.450408,-0.283551,-0.104894
62,-0.458159,-0.323779,-0.125848
63,-0.460340,-0.336867,-0.133166
...,...,...,...
1938,0.000000,0.000000,0.000000
1939,0.000000,0.000000,0.000000
1940,0.000000,0.000000,0.000000


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1884 entries, 59 to 1942
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   corr_px          1884 non-null   float64
 1   corr_log_r(win)  1884 non-null   float64
 2   corr_w_sum       1884 non-null   float64
dtypes: float64(3)
memory usage: 58.9 KB


In [116]:
ticker1_df = pd.read_csv(f'/Users/admin/Desktop/life_web_ui/database/stocks/{ticker1}.csv')
ticker2_df = pd.read_csv(f'/Users/admin/Desktop/life_web_ui/database/stocks/{ticker2}.csv')

In [115]:
ticker1_df(index)

,date_keys,PX_LAST,CHG_PCT_1D,LOG_CHG_PCT_1D,LOG_CHG_PCT_1D_win
0,1,22600.0,-2.7957,-0.028355,-0.026349
1,2,23050.0,1.9912,0.019716,0.019716
2,3,22650.0,-1.7354,-0.017506,-0.017506
3,4,22700.0,0.2208,0.002206,0.002206
4,5,22850.0,0.6608,0.006586,0.006586
...,...,...,...,...,...
2179,2180,23450.0,-1.0549,-0.010605,-0.010605
2180,2181,23000.0,-1.9190,-0.019377,-0.019377
2181,2182,22900.0,-0.4348,-0.004357,-0.004357
2182,2183,22500.0,-1.7467,-0.017621,-0.017621


In [117]:
ticker2_df

,date_keys,PX_LAST,CHG_PCT_1D,LOG_CHG_PCT_1D,LOG_CHG_PCT_1D_win
0,242,37900.0,-36.8333,-0.459393,-0.042111
1,243,27650.0,-27.0449,-0.315326,-0.042111
2,244,23950.0,-13.3816,-0.143658,-0.042111
3,245,25000.0,4.3841,0.042907,0.042907
4,246,23000.0,-8.0000,-0.083382,-0.042111
...,...,...,...,...,...
1938,2180,14430.0,-1.1644,-0.011712,-0.011712
1939,2181,14280.0,-1.0395,-0.010449,-0.010449
1940,2182,14270.0,-0.0700,-0.000700,-0.000700
1941,2183,14190.0,-0.5606,-0.005622,-0.005622


In [82]:
type(eval('[2, 5, 12]'))

list

In [83]:
data = {
    'ID': [1, 2, 3, 4, 5],
    'str_list_column': ["[1, 2, 3, 4, 5]", "[2, 4, 6, 8, 10]", "[3, 6, 9, 12, 15]", "[4, 8, 12, 16, 20]", "[5, 10, 15, 20, 25]"]
}

df = pd.DataFrame(data)

In [84]:
df

,ID,str_list_column
0,1,"[1, 2, 3, 4, 5]"
1,2,"[2, 4, 6, 8, 10]"
2,3,"[3, 6, 9, 12, 15]"
3,4,"[4, 8, 12, 16, 20]"
4,5,"[5, 10, 15, 20, 25]"


In [88]:
df[df['str_list_column'].apply(lambda x: eval(x)[-1] >10)]

,ID,str_list_column
2,3,"[3, 6, 9, 12, 15]"
3,4,"[4, 8, 12, 16, 20]"
4,5,"[5, 10, 15, 20, 25]"
